# Experiment 05-01

In [224]:
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information

## Show the available Platforms

In [225]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:44)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [226]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [227]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [228]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;get_random_numbers;get_random_string;get_single_hash_nonce;mine_nonce


## Initial Tests

### Single Hash

In [229]:
# the plaintext to hash
#plaintext = 'bells hel the quick brown fox jumps over the lazy dog 12356'
plaintext = 'this is a description of the latest block' + '395707976'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

# print the results
print("Plaintext: %s" % plaintext)
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)

assert hexlify(hash_output) == hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest())


Plaintext: this is a description of the latest block395707976
CL result      : b'0000000053d18bbf3e8293e6a3bacda6e35ee7715e6353bf55a86cb9c3b4167b'
Correct result : b'0000000053d18bbf3e8293e6a3bacda6e35ee7715e6353bf55a86cb9c3b4167b'
CL output      : [         0 3213611347 3868426814 2798500515 1910988515 3209913182
 3110905941 2065085635]


### Random String Generation

In [230]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
start = np.uint8(32)
end = np.uint8(126)
length = np.uint32(4)
result = np.zeros(length, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_start = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=start)
cl_end = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=end)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=length)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)

# execute the program
cl_program.get_random_numbers(
    cl_queue, (1,), None,
    cl_seed,
    cl_start,
    cl_end,
    cl_len,
    cl_result)

# get the results
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Seed: {seed}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Seed: 2353436862
Result: [84 65 58 82]
Result String: b'TA:R'


In [231]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
result = np.zeros(shape=16, dtype=np.uint8)
length = np.zeros(shape=(1,), dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, length.nbytes)

# execute the program
cl_program.get_random_string(
    cl_queue, (1,), None,
    cl_seed,
    cl_result,
    cl_len)

# get the results
cl.enqueue_copy(cl_queue, length, cl_len)
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Length: {length}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Length: [6]
Result: [ 53 110  62  70  37  85   0   0   0   0   0   0   0   0   0   0]
Result String: b'5n>F%U\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


## Test get_single_hash_nonce

In [232]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
nonce = np.zeros(shape=16, dtype=np.uint8)
nonce_len = np.zeros(shape=1, dtype=np.uint8)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_hash_output = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_output.nbytes)

# execute the program
cl_program.get_single_hash_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

nonce_str = nonce[:nonce_len[0]].tobytes().decode('UTF-8')

# print the results
print("Seed: %s" % seed)
print("Plaintext: %s" % plaintext)
print("Nonce: %s" % nonce)
print("Nonce Length: %s" % nonce_len[0])
print("Nonce String: %s" % nonce_str)
print("Hash Output: %s" % hash_output)
print("Hash Output string: %s" % hexlify(hash_output))
#print("No Nonce          : %s" % hexlify(hashlib.sha256((plaintext).encode('utf-8')).digest()))
print("Correct result    : %s" % hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest()))

assert hexlify(hash_output) == hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest())


Seed: 3514623740
Plaintext: this is a description of the latest block
Nonce: [92 69 67 56 91  0  0  0  0  0  0  0  0  0  0  0]
Nonce Length: 5
Nonce String: \EC8[
Hash Output: [3100550853 1425789547 3885309443 1264630975 3494420282 1809378304
 1014903885 1850468323]
Hash Output string: b'c5a6ceb86bd2fb54031e95e7bfbc604b3a9f48d000ecd86b4d347e3ce3e74b6e'
Correct result    : b'c5a6ceb86bd2fb54031e95e7bfbc604b3a9f48d000ecd86b4d347e3ce3e74b6e'


## Mine Nonce

In [233]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(64)
nonce = np.zeros(shape=16 * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

# execute the program
cl_program.mine_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_window_size,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

# interpret the results
for i in range(64):
    nonce_str = nonce[i * 16:i * 16 + nonce_len[i]].tobytes().decode('UTF-8')
    hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8')).digest()

    print("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hexlify(hash)))

   0: [15]  $OcpJ'LoC0#QLgk b'c51c2d9076929d03e9b24101f5f9fc1a0468057fecf3198fdf923a380cce419c'
   1: [14]   \=NW5gkO\=MnD/ b'2e47f609c80039f8574718a6d4e10ca4721e0f7f6125a52481931a2fff720522'
   2: [ 4]             UTH+ b'd7670504847a9e9c9a7496c307ad0c0194363b0d6cd1c87060ad3b4f1ff38f10'
   3: [15]  #W5fdv:X;W=@5fd b'614bc57633414b775aca7c75367ac5ea886ef962f4176bf0cc381f8aa60786f6'
   4: [ 6]           4&LhT@ b'7becbedcf2332807c1861789a4f1606ee6e5deb46331e761fe38e3811b7a7e92'
   5: [ 4]             fe^% b'aa491f9788b520805daaaa92fb9f1744c65b6b79d56722276ae88aaec231bdae'
   6: [13]    D=F}fdv;_3'K& b'eb64496dd8d541c3f90756f12aa011ce299e418ebcc2d66884da5f5053f9696c'
   7: [12]     m[?Se$^,/|_+ b'c48e62ad311865b0ccf282f3c32229690304d4a2425872b49517011b9fb1459a'
   8: [14]   |$^-6sEC8\=NW6 b'8308f21da8c2e3e31ca9909920171a6abd602772cbde6712773e2f0bae9e28ef'
   9: [13]    VSe$^,/}fdu5m b'9de848bec43f84be0072b0eb287c89a4367b9a236693c25f6b4ce9dd8487085e'
  10: [ 6]           UN^%\E b'c1ad3e9391